In [5]:
####################################
#
#  ADD THIS TO EVERY COLAB FILE!
#
####################################

#!pip install -q import-ipynb

from google.colab import drive
drive.mount('/content/drive')

#import import_ipynb
import drive.Shareddrives.GPTJ.project.settings as settings

PATH_PROJECT = settings.PATH_PROJECT
PATH_DATA = settings.PATH_DATA

! cd $PATH_PROJECT && pip install -q -r requirements.txt

Mounted at /content/drive
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 143 kB 28.8 MB/s 
     |████████████████████████████████| 55.9 MB 290 kB/s 
     |████████████████████████████████| 717 kB 71.4 MB/s 
     |████████████████████████████████| 163 kB 52.8 MB/s 
     |████████████████████████████████| 7.6 MB 61.8 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 120 kB 57.4 MB/s 
     |████████████████████████████████| 99 kB 10.4 MB/s 


In [6]:
!nvidia-smi

Thu Oct  6 21:17:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel, GPTNeoForCausalLM

In [8]:
# MODEL

#model_name = 'EleutherAI/gpt-neo-1.3B' # CUDA out of memory.

model_name = 'gpt2' # GPU vRAM 3495MiB - 3.5G
#model_name = 'gpt2-medium' # GPU vRAM 7389MiB - 7.3G
#model_name = 'gpt2-large' # GPU vRAM 14797MiB - 14.8G

#model_name = 'gpt2-xl' # CUDA out of memory.


In [14]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).cuda()

#torch.save(model, os.path.join(PATH_DATA, model_name, 'model'))

In [16]:
# Set the random seed to a fixed value to get reproducible results 
torch.manual_seed(42)

prompt = 'Four women'
generated = tokenizer(prompt, return_tensors='pt').input_ids.cuda()

# Generate 
sample_outputs = model.generate(generated, 
    # Use sampling instead of greedy decoding 
    do_sample=True, 
    # Keep only top 50 token with the highest probability
    top_k=50, 
    # Maximum sequence length
    max_length=50,
    #max_new_tokens=100,
    # Keep only the most probable tokens with cumulative probability of 95%
    top_p=0.95, 
    # Changes randomness of generated sequences to 1.9
    temperature=0.0001,
    # Number of sequences to generate                 
    num_return_sequences=3
)

# Print generated descriptions
for i, sample_output in enumerate(sample_outputs): 
    print('{}: {}'.format(i, tokenizer.decode(sample_output, skip_special_tokens=True)).replace('\n', ' '))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Four women have been arrested in connection with the attack, which left at least seven dead and more than 100 injured.  The attack comes as the government is facing criticism for failing to tackle the problem of sexual violence in the country.  The
1: Four women have been arrested in connection with the attack, which left at least seven dead and more than 100 injured.  The attack comes as the government is facing criticism for failing to tackle the problem of sexual violence in the country.  The
2: Four women have been arrested in connection with the attack, which left at least seven dead and more than 100 injured.  The attack comes as the government is facing criticism for failing to tackle the problem of sexual violence in the country.  The


In [11]:
!nvidia-smi

Thu Oct  6 21:17:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    34W /  70W |   1666MiB / 15109MiB |     44%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------